# Approach Tried

* Normalization
* Feature transformation using boxcox for skewed features
* SMOTE for over sampling
* Balancing Ensemble technique
* Stacked Classifier
* New feature addition using existing ones
* Missing values imputation using Most Frequent method specific to each class.
* Various Ensemble models with or without using Balancing technique
* Various different encoding technique’s combination like target encoding , helmert, sum encoder, frequency encoding, OHE etc

Most of the above techniques did not work out so I moved out the code related to them to commented section as much as I was able to recollect.

In [660]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn import preprocessing 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report
from scipy.stats import boxcox

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/finalsub/test-data.csv
/kaggle/input/finalsub/train.csv


In [661]:
def preprocess(df):    
    #Missing Values imputation using most Frequent method 
    #for those columns who are having missing values less than 75%
    
    miss_feat=['Health Indicator','Holding_Policy_Duration','Holding_Policy_Type'] 
    for i in miss_feat:
        df[i]=df[i].fillna(df[i].mode().iat[0])
        
    #Categorical features which are needed to be converted
    cat_feat=['Accomodation_Type','Reco_Policy_Cat','Region_Code' ,'City_Code', 'Reco_Insurance_Type', 'Is_Spouse', 'Health Indicator', 'Holding_Policy_Duration']
    #Converting using label_encoder 
    label_encoder = preprocessing.LabelEncoder() 
    for i in cat_feat:
        df[i] = label_encoder.fit_transform(df[i])
    #Converting using One hot encoding
    df = pd.get_dummies(df, columns=['Holding_Policy_Type'])
    #Normalize
    mx=df['Reco_Policy_Premium'].max()
    mn=df['Reco_Policy_Premium'].min()
    for i in range(len(df['Reco_Policy_Premium'])):
        df['Reco_Policy_Premium'][i]=(df['Reco_Policy_Premium'][i]-mn)/(mx-mn)
    return df
    
#OUTLIER DETECTION
def out_std(s, nstd=3):
    data_mean, data_std = s.mean(), s.std()
    cut_off = data_std * nstd
    lower, upper = data_mean - cut_off, data_mean + cut_off
    #print(lower, upper)
    masked = [True if x < lower or x > upper else False for x in s]
    if masked.count(True)!=0:
        idx=[]
        for i in range(len(masked)):
            if masked[i]==True:
                idx.append(i)
        return idx
    return -1

In [ ]:
df=pd.read_csv('/kaggle/input/finance/train.csv')
X_test=pd.read_csv('/kaggle/input/finance/test-data.csv')

df=df.drop(['ID','Lower_Age'],axis=1)
ID=X_test['ID']
X_test=X_test.drop(['ID','Lower_Age'],axis=1)

In [663]:
# Finding missing Values % 
df.isnull().sum()*100/df.shape[0]

City_Code                   0.000000
Region_Code                 0.000000
Accomodation_Type           0.000000
Reco_Insurance_Type         0.000000
Upper_Age                   0.000000
Lower_Age                   0.000000
Is_Spouse                   0.000000
Health Indicator           22.976691
Holding_Policy_Duration    39.799929
Holding_Policy_Type        39.799929
Reco_Policy_Cat             0.000000
Reco_Policy_Premium         0.000000
Response                    0.000000
dtype: float64

## Adding new Features

In [ ]:
#Region's total no. of positive response  
regn_resp_count=list(df.groupby('Region_Code').sum()['Response'])
regns_idx=df.groupby('Region_Code').sum()['Response'].index

regn_resp=pd.DataFrame({'Region_Code':regns_idx,'regn_resp_count':regn_resp_count})
regn_resp=regn_resp.sort_values('Region_Code')

#Region's total no. of counts and % of positive response 
regn_count=df.groupby('Region_Code').size()
idx=regn_count.index
regn_count=pd.DataFrame({'Region_Code':idx,'regn_counts':list(regn_count)})
regn_count=regn_count.sort_values('Region_Code')
regn_count['regn_resp_count']=regn_resp['regn_resp_count']
regn_count['pct_response_r']=regn_count['regn_resp_count']/regn_count['regn_counts']

df=df.merge(regn_count[['Region_Code','pct_response_r']],how='left')

In [ ]:
# Existing customer based on Holding_Policy_Type and Holding_Policy_Duration
idx_exist_cust=df[df['Holding_Policy_Type'].isnull() & df['Holding_Policy_Duration'].isnull() ].index.tolist()
df['Existing_Cust']=0
for index, row in df.iterrows():
    if int(index) in idx_exist_cust:
        df['Existing_Cust'][index]=1

In [664]:
X_test=preprocess(X_test)
df=preprocess(df)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


# Balancing Dataset
Dividing Data set into 4 batches 

In [665]:
df0=df[df['Response']==0].reset_index(drop=True)
df1=df[df['Response']==1].reset_index(drop=True)

In [666]:
df01=df0[:12209]
df02=df0[12209:24418]
df03=df0[24418:36627]
df04=df0[36627:38673]
df14=df1.sample(2046)

In [667]:
df11=pd.concat([df01,df1])
df12=pd.concat([df02,df1])
df13=pd.concat([df03,df1])
df14=pd.concat([df04,df1])

In [668]:
df11=df11.sample(frac=1)
df12=df12.sample(frac=1)
df13=df13.sample(frac=1)
df14=df14.sample(frac=1)

In [669]:
y1=df11['Response']
X1=df11.drop(['Response'],axis=1)
y2=df12['Response']
X2=df12.drop(['Response'],axis=1)
y3=df13['Response']
X3=df13.drop(['Response'],axis=1)
y4=df14['Response']
X4=df14.drop(['Response'],axis=1)

In [670]:
#Predicting on all 4.

xg1=XGBClassifier(n_estimators=200)
xg1.fit(X1,y1)
y_pred1=xg1.predict_proba(X_test)[:, 1]

xg2=XGBClassifier(n_estimators=200)    
xg2.fit(X2,y2)                         
y_pred2=xg2.predict_proba(X_test)[:, 1]

xg3=XGBClassifier(n_estimators=200)    
xg3.fit(X3,y3)                         
y_pred3=xg3.predict_proba(X_test)[:, 1]
   
xg4=XGBClassifier(n_estimators=200)    
xg4.fit(X4,y4)                         
y_pred4=xg4.predict_proba(X_test)[:, 1]

/opt/conda/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[17:40:14] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:40:17] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:40:20] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[17:40:22] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old be

## Imbalanced model for ensemble having 10% class 0 only

In [671]:
dfus0=df0.sample(1220)
dfus1=df1
dfus=pd.concat([dfus0,dfus1])

yus=dfus['Response']
Xus=dfus.drop(['Response'],axis=1)

xgus=XGBClassifier(n_estimators=200)
xgus.fit(Xus,yus)
y_predus=xgus.predict_proba(X_test)[:, 1]

[17:40:24] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [672]:
#Final probablities from each model
y_final=[]
for i in range(len(y_pred2)):
    count=0
    count=y_pred1[i]+y_pred2[i]+y_pred3[i]+y_pred4[i]+y_predus[i]
    y_final.append(count/5)

In [673]:
#Preparing CSV file for predictions on Xtest
test_pred=pd.DataFrame({'ID':ID,'Response':y_final})
test_pred.to_csv('v15.csv',index=False)

In [674]:
#test_pred[test_pred['Response']>0.5].shape

(18431, 2)

# Commented unused code

 

In [676]:
# #Citys's total no. of positive response  
# city_resp_count=list(df.groupby('City_Code').sum()['Response'])
# cities_idx=df.groupby('City_Code').sum()['Response'].index

# city_resp=pd.DataFrame({'City_Code':cities_idx,'city_resp_count':city_resp_count})
# city_resp=city_resp.sort_values('City_Code')

# #Citys's total no. of counts and % of positive response 
# city_count=df.groupby('City_Code').size()
# idx=city_count.index
# city_count=pd.DataFrame({'City_Code':idx,'city_counts':list(city_count)})
# city_count=city_count.sort_values('City_Code')
# city_count['city_resp_count']=city_resp['city_resp_count']
# city_count['pct_response']=city_count['city_resp_count']/city_count['city_counts']
# df=df.merge(city_count[['City_Code','pct_response']],how='left')

In [678]:
#SEABORN for finding out distribution of data

# fig, axs = plt.subplots(ncols=2)
# fig.tight_layout()
# sns.histplot(data=df0.dropna(), x="pct_response",ax=axs[0])
# sns.histplot(data=df1.dropna(), x="pct_response",ax=axs[1])

In [679]:
#SMOTE for over sampling  
# sm = SMOTE(random_state=2)
# X_train_res, y_train_res = sm.fit_resample(X, y)

In [680]:
#For Outlier removal

# s=set()
# for i in df.columns:
#     idx=out_std(df[i])
#     if idx!=-1:
#         #print(i,idx)
#         s=s|set(idx)
# df=df.drop(list(s))

In [681]:
# Frequency Encoder
#         fq = df.groupby(i).size()/len(df)    
#         # mapping values to dataframe 
#         df.loc[:, "{}_freq_encode".format(i)] = df[i].map(fq)   
#         # drop original column. 
#         df = df.drop([i], axis = 1)

In [682]:
# Missing values imputation using Most Frequent method specific to each class.
# df0=df[df['Response']==0]
# df1=df[df['Response']==1]
# miss_feat=['Health Indicator','Holding_Policy_Duration','Holding_Policy_Type'] 
# for i in miss_feat:
#     df0[i]=df0[i].fillna(df0[i].mode().iat[0])
#     df1[i]=df1[i].fillna(df1[i].mode().iat[0])
# df=pd.concat([df0,df1])

**Author**: Abhishek Soni

**Email**: abhisheksoni@hotmail.com

**Contact number**: +919772169889